In [41]:
%matplotlib inline
import quantstats as qs
import yfinance as yf
import panel as pn
import pandas as pd
import hvplot.pandas
pn.extension()
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import HRPOpt
# extend pandas functionality with metrics, etc.
qs.extend_pandas()
window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)
title = 'Stock'
ticker = pn.widgets.input.TextInput()
# ticker.value = ''
portfolio = None
print(type(portfolio))
init = 0
# input ticker and view stock data

def get_df(ticker, window_size):
    df = yf.Ticker(ticker).history(period="5Y")
    df.reset_index(inplace=True)
    global portfolio
    if ticker is not '' and portfolio is None:
        portfolio = df[['Date', 'Close']]
        portfolio.rename(columns={'Close': ticker}, inplace=True)
    elif ticker is not '':
        portfolio = pd.concat([portfolio, df['Close']], join='inner', axis='columns')
        portfolio.rename(columns={'Close': ticker}, inplace=True)
    
    return df.set_index('Date').rolling(window=window_size).mean().reset_index()
def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return df.hvplot.line('Date', 'Close', grid=True, responsive=True, height=300)

pn.Row(
    pn.Column(title, ticker, window),
    pn.bind(get_plot, ticker, window)
)





<class 'NoneType'>
- : No data found for this date range, symbol may be delisted


Row
    [0] Column
        [0] Markdown(str)
        [1] TextInput()
        [2] IntSlider(end=21, name='Window Size', start=1, value=6)
    [1] ParamFunction(function)

In [45]:
# create portfolio analysis
def get_portfolio_opt(portfolio, window_size):
    portfolio.sort_index(ascending=False, inplace=True)
    portfolio.set_index(keys='Date', inplace=True)
    print(portfolio)
    returns = portfolio.pct_change().dropna()
    hrp = HRPOpt(returns)
    hrp_weights = hrp.optimize()
    performance = hrp.portfolio_performance(verbose=True)
    return performance
get_portfolio_opt(portfolio, window)

                  AAPL         AMD         GME
Date                                          
2022-02-08  174.830002  132.850006  124.290001
2022-02-07  171.660004  128.229996  115.599998
2022-02-04  172.389999  123.669998  102.339996
2022-02-03  172.679993  123.599998  102.339996
2022-02-02  175.616257  120.080002   99.230003
...                ...         ...         ...
2017-02-15   31.985769   12.970000   20.672405
2017-02-14   31.870119   13.300000   20.965170
2017-02-13   31.461765   13.260000   20.932640
2017-02-10   31.185595   13.490000   20.631744
2017-02-09         NaN   13.580000   20.867582

[1259 rows x 3 columns]
Expected annual return: -27.1%
Annual volatility: 25.5%
Sharpe Ratio: -1.14


(-0.27084399783894847, 0.25530957108485175, -1.1391817259458985)